# Aggregating Data with Group by

In this session we will explore data aggregation using pandas.  Aggregation means splitting a dataset up into parts and applying mathematical or other functions to those groups of data.

By the end you will be able to create a monthly time-series showing numbers of referrals, separated by priority level.

## Theory 

You may have come across the concept of data aggregation in a SQL course, and the principle is exactly the same here.

The 'group by' operation splits the data into groups based on entries in one or more columns, so that operations or calculations can be performed on subgroups of the data.

For example, say we wanted to calculate the total number of referrals made in each CCG for some time period. Our dataset looks like this:

| ccg_code | specialty | priority | referrals |
| --- | --- | --- | --- | 
| 00L | Cardiology | Routine | 200 |
| 00L | Urology | Routine | 600 |
| 99C | Cardiology | Urgent | 100 |
| 00L | 2WW | 2 Week Wait | 400 |
| 99C | Neurology | Routine | 900 |

The data contains more information than we need, because we are not interested in the specialty or priority. In which case we want to sum referrals **for each CCG**, summing over the values in other columns. In other words, we want to *group by* CCG and calculate the sum:

| ccg_code | referrals |
| --- | --- | 
| 00L | 1,200 |
| 99C | 1,000 |

## Sections

* [Group by a single column](#item1)
* [Group by multiple columns](#item2)
* [Group by units of time](#item3)
---

In [1]:
# Importing Python packages we are likely to need
import pandas as pd  # useful for reading and manipulating data tables
import matplotlib.pyplot as plt  # useful for plotting data

In [2]:
# We assume input and output data are in your machine's Downloads folder
input_data_path = "~/Downloads/referrals_oct19_dec20.csv"
output_data_path = "~/Downloads/referrals_by_priority.csv"

### Load the data

* Use `pd.read_csv` to read in the data. 
* Then use `pd.to_datetime` to convert the `week_start` column to datetime format, e.g.

```python
df['my_date_column'] = pd.to_datetime(df['my_date_column'])
```

---
<a name="item1">
<h2>Group by a single column</h2>
</a>

First, we'll take a subset of the data which will be easier to work with. Select rows where `Specialty` column has value `'2WW'` (two week wait cancer referrals).
* Use `==` to make a selection mask
* Use `df.loc` to select rows where the mask values are `True`

```python
smaller_df = df.loc[df['column'] == 'value', :]
```

### Sum 2WW referrals, aggregated by CCG
* Use `df.groupby('ccg_code')`
* Select the `referrals` column
* Use the `sum` method

```python
gb = df.groupby('aggregation_column')
summed_values = gb['value_column'].sum()
```

The result of the aggregation operation (i.e. the sum) is a pandas `Series` object. The series index comprises the CCG codes and the values are the summed referrals.

### Visualise aggregation result
Numbers are great, but we like pictures better. Let's visualise the referrals in each CCG with a bar chart. Use your aggregation result from above and `matplotlib`. 

```python
fig, ax = plt.subplots()
ax.bar(df.index, df.values)
```

### Take the average weekly referrals, for each CCG
Sum is not the only aggregation function you can use. Let's try the same `groupby` result you made previously, but this time use `mean` instead of `sum`.

```python
avg_values = gb['value_column'].mean()
```

### Other useful aggregate functions
* `count`
* `median`
* `std` (standard deviation)
* `sem` (standard error of the mean)
* `min` and `max`

You could also define your own aggregate functions. This flexibility is can be extremely powerful.


---
<a name="item2"><h2>
Group by multiple columns
</h2></a>

It's possible to group by multiple columns at once. This means you apply the aggregation function to each subset of values in the columns you group by. For example, if you group by `CCG` and `priority` then use the `sum` function, you will get the total number of referrals per CCG *and* per priority level.

```python
gb_multi_cols = df.groupby(['column_1', 'column_2'])
summed_values = gb_multi_cols['value_colum'].sum()
```
The result is still a pandas `Series` object, but this time the index has two 'levels', one for each aggregation column.

Let's group by both `specialty` and `priority`, then take the sum.

---
<a name="item3"><h2>Group by units of time</h1></a>

You can group by `datetime` values, just like anything else. But sometimes the units of time might be too granular to be useful, e.g. your `datetime` values may be expressed at the level of minutes or seconds, but if your data span a year then you probably don't want to show each individual data point.

Pandas has a special date `Grouper` object, which you provide to `df.groupby`. Its `key` argument lets you specify the column you want to group by and the `freq` argument specifies the desired frequency, where
* `D` means daily
* `W` means weekly
* `M` means monthly

The full list of frequency options can be found <a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases">here</a>.

```python
gb_daily = df.groupby(pd.Grouper(key='column_name', freq='D'))
agg_df = gb_daily['value_column'].sum()
```

Find the total number of referrals by month and then plot the resulting time series, then plot with `df.plot()`. What happened in March 2020?

### Unstacking

We saw above that grouping by multiple columns at once gives us a pandas `Series` object with two index 'levels'. But it is easier to handle the data if one of these levels can be switched around and transformed into multiple *columns* instead. In pandas this is called an `unstack` operation, by analogy with taking a stack of books (like horizontal rows) and rotating them to place on a shelf (like vertical columns).

```python
gb_multi_cols = df.groupby(['column_1', 'column_2'])
summed_values_stacked = gb_multi_cols['value_colum'].sum()
summed_values_unstacked_c1 = summed_values_stacked.unstack('column_1')
```
For example, let's say our dataframe looked like this and then we used `groupby` with `sum`:

In [11]:
# define example data
example_data = {
    "ccg_code": ["00L", "00L", "99C", "00L", "99C"],
    "priority": ["Routine", "Routine", "Urgent", "2 Week Wait", "Routine"],
    "referrals": ["2", "6", "1", "4", "9"]
}
example = pd.DataFrame(example_data)
example

,ccg_code,priority,referrals
0,00L,Routine,2
1,00L,Routine,6
2,99C,Urgent,1
3,00L,2 Week Wait,4
4,99C,Routine,9


In [12]:
# aggregate example data
gb_example_stacked = example.groupby(['ccg_code', 'priority']).sum()
gb_example_stacked

referrals
ccg_code priority             
00L      2 Week Wait         4
         Routine            26
99C      Routine             9
         Urgent              1

We can make the `priority` level into columns, like this:

In [13]:
gb_example_stacked.unstack('priority')

referrals               
priority 2 Week Wait Routine Urgent
ccg_code                           
00L                4      26    NaN
99C              NaN       9      1

Notice there's missing data (`NaN`) for some elements, since those combinations weren't present in the original data.

Now let's group by both month and priority and take the total number of referrals for each, then unstack the priority level, so that months are rows and priorities are columns. This will allow us to plot the time series for each priority simultaneously.

E.g. result of the following will have rows for each date and columns for each value of `other_column`:

```python
gb_date_and_col = df.groupby([pd.Grouper(key='date_column', freq='M'), 'other_column'])
agg_df = gb_monthly_priority['value'].sum()
unstacked = agg_df.unstack('other_column')
```

Then we can plot these all together with `unstacked.plot()`.

Finally, save your output time series using the `output_data_path` defined above.

```python
timeseries_df.to_csv(output_data_path)
```